<a href="https://colab.research.google.com/github/fmaignacio/observatorio-tere/blob/main/transcricoes_todos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gera a base de dados para construção do Dashboard

In [ ]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
"""
TESTE RÁPIDO - Processa apenas 3 arquivos para diagnóstico
"""

import os
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Caminho da pasta
pasta = '/content/drive/MyDrive/observatorio_teresopolis/TranscriçõesSessoesTeresopolis'

print("="*70)
print("🔍 DIAGNÓSTICO DA PASTA")
print("="*70)

# Verifica se pasta existe
if not os.path.exists(pasta):
    print(f"❌ Pasta não existe: {pasta}")
    print("\n💡 Tente listar as pastas disponíveis:")
    print("!ls '/content/drive/MyDrive/'")
else:
    print(f"✅ Pasta existe: {pasta}\n")

    # Lista todos os arquivos
    todos_arquivos = os.listdir(pasta)
    print(f"📁 Total de arquivos: {len(todos_arquivos)}\n")

    # Filtra arquivos .txt
    arquivos_txt = [f for f in todos_arquivos if f.endswith('.txt')]
    print(f"📄 Arquivos .txt: {len(arquivos_txt)}\n")

    if len(arquivos_txt) == 0:
        print("❌ Nenhum arquivo .txt encontrado!")
        print("\nTodos os arquivos na pasta:")
        for i, arquivo in enumerate(todos_arquivos[:20], 1):
            print(f"  {i}. {arquivo}")
        if len(todos_arquivos) > 20:
            print(f"  ... e mais {len(todos_arquivos) - 20} arquivos")
    else:
        print("✅ Arquivos .txt encontrados:")
        for i, arquivo in enumerate(arquivos_txt[:10], 1):
            tamanho = os.path.getsize(os.path.join(pasta, arquivo))
            print(f"  {i}. {arquivo} ({tamanho:,} bytes)")

        if len(arquivos_txt) > 10:
            print(f"  ... e mais {len(arquivos_txt) - 10} arquivos")

        # Teste de leitura dos primeiros 3 arquivos
        print("\n" + "="*70)
        print("📖 TESTE DE LEITURA (3 primeiros arquivos)")
        print("="*70 + "\n")

        for i, arquivo in enumerate(arquivos_txt[:3], 1):
            print(f"[{i}/3] Testando: {arquivo}")
            try:
                caminho = os.path.join(pasta, arquivo)
                with open(caminho, 'r', encoding='utf-8') as f:
                    conteudo = f.read()

                # Mostra primeiros 500 caracteres
                print(f"  ✅ Lido com sucesso ({len(conteudo):,} caracteres)")
                print(f"  📝 Início do arquivo:")
                print("  " + "-"*66)
                print("  " + conteudo[:500].replace('\n', '\n  '))
                print("  " + "-"*66)
                print()

            except Exception as e:
                print(f"  ❌ Erro ao ler: {e}\n")

print("\n" + "="*70)
print("✅ Diagnóstico concluído!")
print("="*70)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 DIAGNÓSTICO DA PASTA
✅ Pasta existe: /content/drive/MyDrive/observatorio_teresopolis/TranscriçõesSessoesTeresopolis

📁 Total de arquivos: 64

📄 Arquivos .txt: 63

✅ Arquivos .txt encontrados:
  1. video-oxLspHRq6fo-ytranscript.txt (83,806 bytes)
  2. video-EeqUDqM_VzE-ytranscript.txt (53,382 bytes)
  3. video-KlVAXPja2jI-ytranscript.txt (8,230 bytes)
  4. video-JPVf0YsX_6A-ytranscript.txt (76,866 bytes)
  5. video-3RdIXI6Cc18-ytranscript.txt (19,163 bytes)
  6. video-HCqLAjXnDAY-ytranscript.txt (85,413 bytes)
  7. video-o9irVb9EFb8-ytranscript.txt (104,558 bytes)
  8. video-hw1VWiNyI-0-ytranscript.txt (59,804 bytes)
  9. video-MTRFUgpuoGw-ytranscript.txt (41,288 bytes)
  10. video-xdk6i0-UGno-ytranscript.txt (53,197 bytes)
  ... e mais 53 arquivos

📖 TESTE DE LEITURA (3 primeiros arquivos)

[1/3] Testando: video-oxLspHRq6fo-ytranscript.txt
  ✅ Lido com suce

In [ ]:
"""
Extrai data do TÍTULO do vídeo no YouTube (web scraping)
Sem necessidade de API Key
"""

import spacy
import re
import pandas as pd
import os
import requests
from google.colab import drive
from bs4 import BeautifulSoup
import time

print("🚀 Sistema com Extração de Título do YouTube")

drive.mount('/content/drive')
nlp = spacy.load("pt_core_news_lg")

pasta_transcricoes = '/content/drive/MyDrive/observatorio_teresopolis/TranscriçõesSessoesTeresopolis'

VEREADORES_MASTER = {
    "Amanda": ["Amanda", "professora Amanda"],
    "André do Gás": ["André do Gás", "André"],
    "Bruninho Almeida": ["Bruninho Almeida", "Bruninho"],
    "Cacau Repórter": ["Cacau Repórter", "Cacau"],
    "Caio Perfister": ["Caio Perfister", "Caio Perfiste", "Caio"],
    "Calé": ["Calé"],
    "Dudu do Resgate": ["Dudu do Resgate", "Dudo Resgate", "Dudu"],
    "Diego Barbosa": ["Diego Barbosa", "Diego"],
    "Fabinho Filé": ["Fabinho Filé", "Fabinho"],
    "Fidel Faria": ["Fidel Faria", "Fidel"],
    "Igor Faraco": ["Igor Faraco", "Igor"],
    "João Miguel": ["João Miguel", "João"],
    "Luciano Santos": ["Luciano Santos", "Luciano"],
    "Márcia Valentim": ["Márcia Valentim", "Márcia"],
    "Marcos Rangel": ["Marcos Rangel", "Rangel"],
    "Maurício Lopes": ["Maurício Lopes", "Maurício"],
    "Paulinho Nogueira": ["Paulinho Nogueira", "Paulinho"],
    "Sandrinho": ["Sandrinho"],
    "Totó": ["Totó", "Totó Online"],
    "Vitinho Nogueira": ["Vitinho Nogueira", "Vitinho"],
    "Érica Marra": ["Érica Marra", "Érica"]
}

def extrair_video_id(texto, nome_arquivo):
    """Extrai ID do vídeo do YouTube"""
    # Tenta extrair da URL nas primeiras linhas
    primeiras_linhas = '\n'.join(texto.split('\n')[:5])

    match = re.search(r'youtube\.com/watch\?v=([a-zA-Z0-9_-]{11})', primeiras_linhas)
    if match:
        return match.group(1)

    # Tenta extrair do nome do arquivo
    match = re.search(r'video-([a-zA-Z0-9_-]{11})', nome_arquivo)
    if match:
        return match.group(1)

    return None

def buscar_titulo_youtube(video_id):
    """Busca o título do vídeo no YouTube via web scraping"""
    if not video_id:
        return None

    url = f"https://www.youtube.com/watch?v={video_id}"

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            # Procura pelo título na página HTML
            match = re.search(r'"title":"([^"]+)"', response.text)
            if match:
                titulo = match.group(1)
                # Decodifica caracteres Unicode
                titulo = titulo.encode().decode('unicode_escape')
                return titulo
    except Exception as e:
        print(f"      Erro ao buscar título: {e}")

    return None

def extrair_data_do_titulo(titulo):
    """Extrai data do título do vídeo"""
    if not titulo:
        return None

    # Padrões comuns em títulos:
    # "Sessão 14/08/2025"
    # "14-08-2025"
    # "14.08.2025"
    # "14/08/25"

    # Formato: DD/MM/YYYY ou DD-MM-YYYY ou DD.MM.YYYY
    match = re.search(r'(\d{2})[-/.](\d{2})[-/.](\d{4})', titulo)
    if match:
        dia, mes, ano = match.groups()
        ano_int = int(ano)
        if 2024 <= ano_int <= 2026:
            return f"{ano}-{mes}-{dia}"

    # Formato: DD/MM/YY
    match = re.search(r'(\d{2})[-/.](\d{2})[-/.](\d{2})\b', titulo)
    if match:
        dia, mes, ano = match.groups()
        ano_completo = f"20{ano}"
        ano_int = int(ano_completo)
        if 2024 <= ano_int <= 2026:
            return f"{ano_completo}-{mes}-{dia}"

    # Formato: "DD de MMMM de YYYY"
    match = re.search(
        r'(\d{1,2})\s+de\s+(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s+de\s+(202[4-6])',
        titulo,
        re.IGNORECASE
    )
    if match:
        dia, mes_nome, ano = match.groups()
        meses = {
            'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4,
            'maio': 5, 'junho': 6, 'julho': 7, 'agosto': 8,
            'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
        }
        mes_num = meses[mes_nome.lower()]
        return f"{ano}-{mes_num:02d}-{int(dia):02d}"

    return None

def extrair_data_do_corpo(texto):
    """Fallback: Extrai data do corpo da transcrição"""
    texto_busca = texto[:5000]

    matches = re.finditer(
        r'(\d{1,2})\s+de\s+(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s+de\s+(202[4-6])',
        texto_busca,
        re.IGNORECASE
    )

    meses = {
        'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4,
        'maio': 5, 'junho': 6, 'julho': 7, 'agosto': 8,
        'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
    }

    for match in matches:
        dia, mes_nome, ano = match.groups()
        mes_num = meses[mes_nome.lower()]
        ano_int = int(ano)
        dia_int = int(dia)

        if 1 <= dia_int <= 31 and 2024 <= ano_int <= 2026:
            return f"{ano_int}-{mes_num:02d}-{dia_int:02d}"

    return None

def extrair_autor_especifico(texto_contexto, pl_numero):
    """Extrai autor específico do PL"""
    padrao_pl_exato = rf'(?:Projeto\s+de\s+Lei|PL)\s+(?:n[úº]mero|n[úº]|nº)?\s*{re.escape(pl_numero)}'
    match_pl = re.search(padrao_pl_exato, texto_contexto, re.IGNORECASE)

    if not match_pl:
        return None

    posicao_pl = match_pl.start()
    contexto_pos_pl = texto_contexto[posicao_pl:posicao_pl + 500]

    for oficial, variacoes in VEREADORES_MASTER.items():
        for variacao in variacoes:
            padrao1 = rf'(?:de\s+autoria|autor).*?(?:vereador|vereadora).*?\b{re.escape(variacao)}\b'
            if re.search(padrao1, contexto_pos_pl, re.IGNORECASE):
                return oficial

    for oficial, variacoes in VEREADORES_MASTER.items():
        for variacao in variacoes:
            padrao2 = rf'vereador[a]?\s+{re.escape(variacao)}\b'
            match = re.search(padrao2, contexto_pos_pl[:200], re.IGNORECASE)
            if match and match.start() < 150:
                return oficial

    return None

def determinar_status(contexto, pl_numero):
    """Determina status do PL"""
    contexto_lower = contexto.lower()

    if re.search(rf'{re.escape(pl_numero)}.*?(?:foi\s+)?aprovado', contexto_lower, re.DOTALL):
        return 'Aprovado (Votação Simbólica)'
    if re.search(r'em\s+votação.*?aprovado', contexto_lower):
        return 'Aprovado (Votação Simbólica)'
    if 'rejeitado' in contexto_lower:
        return 'Rejeitado'
    if re.search(r'encaminhad[oa]s?\s+(?:para|à|às)\s+(?:comiss[ãõ]|ccj)', contexto_lower):
        return 'Encaminhado para Comissão'
    if re.search(r'em\s+discuss[ãa]o', contexto_lower):
        return 'Em Discussão'

    return 'Não identificado'

def extrair_presentes(texto):
    """Extrai vereadores presentes"""
    bloco = re.search(r'chamada\s+dos\s+vereadores.*?(?:Questão|Peço|execução|hino)',
                     texto, re.DOTALL | re.IGNORECASE)
    if not bloco:
        return []

    presentes = []
    for oficial, variacoes in VEREADORES_MASTER.items():
        for variacao in variacoes:
            if re.search(r'\b' + re.escape(variacao) + r'\b', bloco.group(0), re.IGNORECASE):
                presentes.append(oficial)
                break
    return presentes

# ===== PROCESSAMENTO =====

print(f"\n📂 Pasta: {pasta_transcricoes}\n")

dados = []
stats = {
    'processados': 0,
    'sem_data': 0,
    'data_corpo': 0,
    'data_titulo': 0,
    'pls_total': 0,
    'pls_sem_autor': 0,
    'datas_encontradas': {}
}

arquivos = [f for f in os.listdir(pasta_transcricoes) if f.endswith('.txt')]
print(f"📄 {len(arquivos)} arquivos encontrados\n")

for idx, arq in enumerate(arquivos, 1):
    try:
        with open(os.path.join(pasta_transcricoes, arq), 'r', encoding='utf-8') as f:
            conteudo = f.read()

        # Estratégia 1: Tenta extrair do corpo
        data = extrair_data_do_corpo(conteudo)
        origem = "corpo"

        # Estratégia 2: Se não achou, busca no título do YouTube
        if not data:
            video_id = extrair_video_id(conteudo, arq)
            if video_id:
                print(f"[{idx}/{len(arquivos)}] {arq[:40]}... 🔍 Buscando título YouTube...", end=" ")
                titulo = buscar_titulo_youtube(video_id)
                if titulo:
                    print(f"\n      Título: {titulo[:80]}")
                    data = extrair_data_do_titulo(titulo)
                    origem = "título"
                    time.sleep(0.5)  # Pequena pausa para não sobrecarregar YouTube

        if not data:
            print(f"[{idx}/{len(arquivos)}] {arq[:40]}... ⚠️ Sem data")
            stats['sem_data'] += 1
            continue

        if origem == "corpo":
            stats['data_corpo'] += 1
        else:
            stats['data_titulo'] += 1

        if data not in stats['datas_encontradas']:
            stats['datas_encontradas'][data] = 0
        stats['datas_encontradas'][data] += 1

        presentes = extrair_presentes(conteudo)

        # Processa PLs
        padrao_pl = r'(?:Projeto\s+de\s+Lei|PL)\s+(?:n[úº]mero|n[úº]|nº)?\s*(\d{1,3}/202[4-6])'
        pls_processados = set()
        pls_encontrados = 0

        for match in re.finditer(padrao_pl, conteudo, re.IGNORECASE):
            pl_numero = match.group(1)

            if pl_numero in pls_processados:
                continue

            pls_processados.add(pl_numero)
            posicao = match.start()

            inicio = max(0, posicao - 1000)
            fim = min(len(conteudo), posicao + 2000)
            contexto = conteudo[inicio:fim]

            autor = extrair_autor_especifico(contexto, pl_numero)

            if not autor:
                stats['pls_sem_autor'] += 1
                continue

            status = determinar_status(contexto, pl_numero)

            dados.append({
                'Data Sessão': data,
                'PL': pl_numero,
                'Autor': autor,
                'Status': status,
                'Votos': 'N/A',
                'Presentes': ', '.join(presentes) if presentes else 'Chamada não identificada',
                'Fonte': arq
            })

            pls_encontrados += 1
            stats['pls_total'] += 1

        print(f"[{idx}/{len(arquivos)}] {arq[:40]}... ✅ {data} ({origem}) | {pls_encontrados} PLs")
        stats['processados'] += 1

    except Exception as e:
        print(f"[{idx}/{len(arquivos)}] {arq[:40]}... ❌ {e}")

print("\n" + "="*70)
print("📊 RESULTADOS FINAIS")
print("="*70)
print(f"✅ Arquivos processados: {stats['processados']}")
print(f"   📄 Datas do corpo: {stats['data_corpo']}")
print(f"   🎬 Datas do título YouTube: {stats['data_titulo']}")
print(f"⚠️  Arquivos sem data: {stats['sem_data']}")
print(f"📋 PLs extraídos: {stats['pls_total']}")
print(f"⚠️  PLs sem autor: {stats['pls_sem_autor']}")

print("\n📅 DISTRIBUIÇÃO DE DATAS:")
for data, qtd in sorted(stats['datas_encontradas'].items()):
    data_formatada = pd.to_datetime(data).strftime('%d/%m/%Y')
    print(f"   {data_formatada}: {qtd} arquivos")

if dados:
    df = pd.DataFrame(dados)
    df['Data Sessão'] = pd.to_datetime(df['Data Sessão'])

    antes = len(df)
    df = df.drop_duplicates(subset=['PL', 'Fonte']).reset_index(drop=True)
    depois = len(df)

    if antes > depois:
        print(f"\n🔄 Duplicatas removidas: {antes - depois}")

    df = df.sort_values('Data Sessão')

    print(f"\n✅ Base final: {len(df)} registros")
    print(f"📅 Período: {df['Data Sessão'].min():%d/%m/%Y} até {df['Data Sessão'].max():%d/%m/%Y}")
    print(f"📋 PLs únicos: {df['PL'].nunique()}")

    print("\n🏆 Top 10 Autores:")
    for i, (autor, qtd) in enumerate(df['Autor'].value_counts().head(10).items(), 1):
        print(f"   {i:2}. {autor:20} {qtd:3} PLs")

    print("\n📊 Status:")
    for status, qtd in df['Status'].value_counts().items():
        pct = (qtd/len(df))*100
        print(f"   • {status:30} {qtd:3} ({pct:5.1f}%)")

    print("\n📅 PLs por Mês:")
    df['Mes_Ano'] = df['Data Sessão'].dt.to_period('M')
    pls_mes = df.groupby('Mes_Ano').size().sort_index()
    for mes, qtd in pls_mes.items():
        print(f"   {mes}: {qtd} PLs")
    df = df.drop('Mes_Ano', axis=1)

    completos = len(df[
        (df['Status'] != 'Não identificado') &
        (df['Presentes'] != 'Chamada não identificada')
    ])
    qualidade = (completos/len(df))*100

    print(f"\n🎯 Qualidade: {qualidade:.1f}%")

    # Salva
    csv_path = '/content/drive/MyDrive/base_observatorio_teresopolis_COMPLETA.csv'
    excel_path = '/content/drive/MyDrive/base_observatorio_teresopolis_COMPLETA.xlsx'

    df.to_csv(csv_path, index=False)

    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='PLs', index=False)

        stats_data = {
            'Métrica': [
                'Total de Registros', 'PLs Únicos', 'Autores Únicos',
                'Data Inicial', 'Data Final', 'Período (dias)',
                'Aprovados', 'Em Discussão', 'Encaminhados',
                'Não Identificados', 'Qualidade (%)'
            ],
            'Valor': [
                len(df), df['PL'].nunique(), df['Autor'].nunique(),
                df['Data Sessão'].min().strftime('%d/%m/%Y'),
                df['Data Sessão'].max().strftime('%d/%m/%Y'),
                (df['Data Sessão'].max() - df['Data Sessão'].min()).days,
                len(df[df['Status'].str.contains('Aprovado', na=False)]),
                len(df[df['Status'] == 'Em Discussão']),
                len(df[df['Status'].str.contains('Encaminhado', na=False)]),
                len(df[df['Status'] == 'Não identificado']),
                f"{qualidade:.1f}%"
            ]
        }
        pd.DataFrame(stats_data).to_excel(writer, sheet_name='Resumo', index=False)

    print(f"\n💾 Arquivos salvos:")
    print(f"   • {csv_path}")
    print(f"   • {excel_path}")

    print("\n" + "="*70)
    display(df.head(30))

else:
    print("\n❌ Nenhum dado extraído")

print("\n✅ PROCESSAMENTO CONCLUÍDO!")

🚀 Sistema com Extração de Título do YouTube
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📂 Pasta: /content/drive/MyDrive/observatorio_teresopolis/TranscriçõesSessoesTeresopolis

📄 63 arquivos encontrados

[1/63] video-oxLspHRq6fo-ytranscript.txt... 🔍 Buscando título YouTube... 
      Título: SessÃ£o da CÃ¢mara de TeresÃ³polis - 12/08/2025
[1/63] video-oxLspHRq6fo-ytranscript.txt... ✅ 2025-08-12 (título) | 8 PLs
[2/63] video-EeqUDqM_VzE-ytranscript.txt... 🔍 Buscando título YouTube... 
      Título: SessÃ£o da CÃ¢mara de TeresÃ³polis - 19/08/2025
[2/63] video-EeqUDqM_VzE-ytranscript.txt... ✅ 2025-08-19 (título) | 8 PLs
[3/63] video-KlVAXPja2jI-ytranscript.txt... ✅ 2025-09-04 (corpo) | 5 PLs
[4/63] video-JPVf0YsX_6A-ytranscript.txt... ✅ 2024-02-23 (corpo) | 12 PLs
[5/63] video-3RdIXI6Cc18-ytranscript.txt... ✅ 2025-09-11 (corpo) | 7 PLs
[6/63] video-HCqLAjXnDAY-ytranscript.txt... ✅ 2025-08-18 (corpo) | 9 P

,Data Sessão,PL,Autor,Status,Votos,Presentes,Fonte
32,2024-02-23,247/2025,Bruninho Almeida,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
31,2024-02-23,237/2025,Bruninho Almeida,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
30,2024-02-23,236/2025,Caio Perfister,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
29,2024-02-23,246/2025,Caio Perfister,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
28,2024-02-23,226/2025,Caio Perfister,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
27,2024-02-23,215/2025,Caio Perfister,Não identificado,N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
26,2024-02-23,265/2025,Maurício Lopes,Aprovado (Votação Simbólica),N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
25,2024-02-23,263/2025,Marcos Rangel,Aprovado (Votação Simbólica),N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
24,2024-02-23,262/2025,Marcos Rangel,Aprovado (Votação Simbólica),N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt
23,2024-02-23,259/2025,Luciano Santos,Aprovado (Votação Simbólica),N/A,"Amanda, André do Gás, Cacau Repórter, Caio Per...",video-JPVf0YsX_6A-ytranscript.txt



✅ PROCESSAMENTO CONCLUÍDO!


TESTES

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_lg")

texto = "Projeto de Lei nº 123/2024 de autoria de Amanda foi aprovado em votação simbólica."
doc = nlp(texto)
for ent in doc.ents:
    print(ent.text, ent.label_)


Projeto de Lei MISC
Amanda PER


In [ ]:
import unittest

# Exemplo simples de função (substitua pelas suas!)
def extrair_data(texto):
    # Coloque aqui sua lógica de extração de data
    import re
    m = re.search(r"(\d{2})/(\d{2})/(\d{4})", texto)
    if m:
        return f"{m.group(3)}-{m.group(2)}-{m.group(1)}"
    return None

def extrair_pl(texto):
    import re
    m = re.search(r"Projeto de Lei n[º°] (\d+/\d{4})", texto)
    if m:
        return m.group(1)
    return None

def extrair_blocos_votacao(texto):
    import re
    matches = list(re.finditer(r'(votação do projeto.*?)(aprovado|rejeitado|encaminhado para comissão)', texto, re.DOTALL | re.IGNORECASE))
    blocos = [m.group(1) for m in matches]
    return blocos

class TestExtracaoObservatorio(unittest.TestCase):
    def test_extrair_data(self):
        texto = "Ata da sessão realizada em 02/10/2025 no plenário."
        self.assertEqual(extrair_data(texto), "2025-10-02")

    def test_extrair_pl(self):
        texto = "Foi discutido o Projeto de Lei nº 123/2024 na sessão de hoje."
        self.assertEqual(extrair_pl(texto), "123/2024")

    def test_extrair_blocos_votacao(self):
        texto = ("Iniciando votação do projeto de lei nº 123/2024. "
                 "Discussão... aprovado. "
                 "Em seguida votação do projeto de lei nº 456/2025. rejeitado.")
        blocos = extrair_blocos_votacao(texto)
        self.assertEqual(len(blocos), 2)
        self.assertIn("votação do projeto de lei nº 123/2024.", blocos[0])

unittest.main(argv=[''], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.017s

OK
